# Quantum Teleportation Demonstrator
### --- "now with (simulated) real quantum magic!"

Let us import some important stuff!

In [8]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

import numpy as np
from qiskit import *
import qiskit
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from math import pi, sqrt
from qiskit.visualization import plot_histogram

import matplotlib
import matplotlib.pyplot as plt

qiskit.__qiskit_version__

ibmqfactory.load_account:WARNING:2020-06-26 14:53:33,132: Credentials are already in use. The existing account in the session will be replaced.


{'qiskit-terra': '0.14.2',
 'qiskit-aer': '0.5.2',
 'qiskit-ignis': '0.3.2',
 'qiskit-ibmq-provider': '0.7.2',
 'qiskit-aqua': '0.7.3',
 'qiskit': '0.19.5'}

Set up backends `statevector_simulator` and `qasm_simulator`. Initialize qubits and classical bits for Alice and Bob.

In [9]:
q_backend = Aer.get_backend("qasm_simulator")
s_backend = Aer.get_backend("statevector_simulator")
shots = 1000

# Alice - two qubits, two classical bits
q_a0 = QuantumRegister(1, "a0")
q_a1 = QuantumRegister(1, "a1")
c_a0 = ClassicalRegister(1, "ca0")
c_a1 = ClassicalRegister(1, "ca1")

# Bob - one qubit, one classical bit
q_b0 = QuantumRegister(1, "b0")
c_b0 = ClassicalRegister(1, "cb0")

Get input for a general quantum state by choosing two random numbers from 0 to 100.

In [16]:
A = int(input("please enter a number between 0 and 100  A: "))

please enter a number between 0 and 100  A: 76


Let us have a look at the quantum state defined by your input and check the initializer by simulating with the statevector simulator.

In [17]:
A_norm = A/100
alpha = A_norm
beta = sqrt(1-alpha**2)
qc = QuantumCircuit(q_a0, c_a0, name="some_state")
input_state = [alpha, beta]
qc.initialize(input_state, q_a0)
job = execute(qc, backend=s_backend)
result = job.result()
sim_state = result.get_statevector()
A_out = int(round(abs(sim_state[0])*100))
print("Input statevector:     {}".format(input_state))
print("Simulated statevector: {}".format(sim_state))
print("Revovered number:      {}".format(A_out))

Input statevector:     [0.76, 0.6499230723708769]
Simulated statevector: [0.76      +0.j 0.64992307+0.j]
Revovered number:      76


### Let the teleportation begin

Hands-On: Prepare the quantum teleportation circuit.

In [6]:
qc = QuantumCircuit(q_a0, q_a1, q_b0, c_a0, c_a1, c_b0, name="teleportation")

# Prepare the initial state
qc.initialize(input_state, q_a0)

#qc.barrier(q_a0, q_a1, q_b0)  # for better drawing

# Prepare an entangled pair (EPR pair) between Alice and Bob
qc.h(1)
qc.cx(q_a1, q_b0)

qc.barrier(q_a0, q_a1, q_b0)  # for better drawing

# Alice to send qubits through CNOT gate
qc.cx(q_a0, q_a1)

qc.barrier(q_a0, q_a1, q_b0)  # for better drawing

# Alice to send first qubit through Hadamard gate
qc.h(0)

qc.barrier(q_a0, q_a1, q_b0)  # for better drawing

# Alice measures her qubits
qc.measure(q_a0, c_a0)
qc.measure(q_a1, c_a1)

# Transmit results on classical channel

qc.barrier(q_a0, q_a1, q_b0)  # for better drawing

# Apply a correction at Bob
qc.z(q_b0).c_if(c_a0, 1)
qc.x(q_b0).c_if(c_a1, 1)

# Measure quantum state by Bob
qc.measure(q_b0, c_b0)

#qc.draw('mpl')

Simulate the teleportation circuit with the qasm simulator and sum over Alice's qubits to get Bob's result of the measurement.

Convert counts into statevector amplitudes and recover teleported number.

In [18]:
job = execute(qc, backend=q_backend, shots=shots)
result = job.result()
counts = result.get_counts(qc)

bob = {}
bob['0'] = 0
bob['1'] = 0
for key, val in counts.items():
    bob[key[0]] += val
#print("Bob's counts:      {}".format(bob))

#plot_histogram(bob, title = "Probability histogram of teleported state")

In [19]:
sim_state[0] = sqrt(bob['0']/shots)
sim_state[1] = sqrt(bob['1']/shots)
A_out = int(round(abs(sim_state[0])*100))
print("Input statevector:     {}".format(input_state))
print("Simulated statevector: {}".format(sim_state))
print("Revovered number:      {}".format(A_out))

Input statevector:     [0.76, 0.6499230723708769]
Simulated statevector: [1.+0.j 0.+0.j]
Revovered number:      100
